In [10]:
# Dependencies
import pandas as pd
import requests as req
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver

In [11]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [12]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
# Visit url for NASA Mars News -- Latest News
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, "html.parser")

# Extract article title and paragraph text
article = soup.find("div", class_='list_text')
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_ ="article_teaser_body").text
print(news_title)
print(news_p)

AttributeError: 'NoneType' object has no attribute 'find'

In [14]:
# Visit url for JPL Featured Space Image
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

# Go to 'FULL IMAGE'
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

# Go to 'more info'
browser.click_link_by_partial_text('more info')

# Parse HTML with Beautiful Soup
html = browser.html
image_soup = bs(html, 'html.parser')

# Scrape the URL
feat_img_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://www.jpl.nasa.gov{feat_img_url}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17550_hires.jpg


In [17]:
#url to twitter page for mars weather
tweet_url = "https://twitter.com/marswxreport?lang=en"
tweet_html_content = req.get(tweet_url).text
soup = bs(tweet_html_content, "lxml")
tweet_list = soup.find_all('div', class_="js-tweet-text-container")

#empty list to hold tweet we are going to keep, used to strip useless content from string
holds_tweet = []

# Loop that scans every tweet and searches specifically for those that have weather info
for tweets in tweet_list: 
    tweet_body = tweets.find('p').text
    if 'InSight' and 'sol' in tweet_body:
        holds_tweet.append(tweet_body)
        #break statement to only print the first weather tweet found
        break
    else: 
        #if not weather related skip it and try again
        pass
    
#cleaned up tweet removes unncessary link to twitter image included in string, :-26 removes the last 26 characters which is the length of the img url
#after reviewing several links they all appear to work with the value of -26
mars_weather = ([holds_tweet[0]][0][:-26])
tweet_img_link = ([holds_tweet[0]][0][-26:])
print(f"{mars_weather}: {tweet_img_link}")

InSight sol 453 (2020-03-05) low -95.1ºC (-139.1ºF) high -10.8ºC (12.6ºF)
winds from the SSW at 6.0 m/s (13.3 mph) gusting to 21.4 m/s (47.9 mph)
pressure at 6.30 hPa: pic.twitter.com/8nEk9RxzXk


In [41]:
# Use Pandas to scrape the table containing facts about Mars
table = pd.read_html(facts_url)
mars_facts = table[0]
# Rename columns
mars_facts.columns = ['Description','Info']

In [42]:
mars_facts

,Description,Info
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [43]:
# Use Pandas to convert the data to a HTML table string
mars_facts.to_html('table.html')

In [44]:
#Save html to folder and show as html table string
mars_df = mars_facts.to_html()
print(mars_df)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Info</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
     

In [55]:
#Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html

In [56]:
soup = bs(html, 'html.parser')

hemisphere_image_urls = []

In [57]:
results = soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

In [58]:
# Iterate through each image
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})

# Print image title and url
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
